<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Импорт и изучение данных:

In [1]:
# Импортирую сразу все библиотеки, которые понадобятся в этом проекте:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import r2_score

In [4]:
# Прочитаем файл:
df = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

Изучим данные:

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [6]:
df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [7]:
df.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Вывод: тип данных в столбце "возраст" не совсем корректный, можно было бы поменять на инт, но у нас в этом нет необходимости.

## Умножение матриц:

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Основные свойства матриц, необходимые для доказательства:

$$
1. A A^-1 = E
$$
$$
2. (A B)^T = B^T A^T
$$
$$
3. (AB)^-1 = B^-1 A^-1
$$

Дано условие:
$$
Х' = XP 
$$

Нужно доказать, что:
$$
a = Xw = X'w' = a'
$$

Подставлю под формулу весов новую матрицу и проведу математические преобразования:

$$
a' = XP((XP)^T XP)^-1 (XP)^T y = XP (P^TX^T XP)^-1 (XP)^T y = XPP^-1(P^TX^TX)^-1 (XP)^T y = XE (P^TX^TX)^-1 (XP)^T y = X (P^TX^TX)^-1 (XP)^T y = X (X^TX)^-1 (P^T)^-1 P^T X^T y = X(X^T X)^-1X^Ty 
$$

Все P сократились по правилу PP^-1 = E, а так как Е - это единичная матрица, как умножение на 1Б можно сократить.

$$
a = X(X^T X)^-1X^Ty = a'
$$

Что значит: качество линейной регрессии не изменится, если признаки умножить на обратимую матрицу.

## Алгоритм преобразования:

Для шифрования данных застрахованных умножим матрицу с данными на радомную обратимую матрицу. Обоснованием является глава 2(выше) и свойство умножения матриц.

**Алгоритм**
1. Создать рандомную матрицу
2. Проверить ее на обратимость
3. Умножить данные на эту новую матрицу
4. Сравнить метрики качества MSE и R2 до и после преобразования данных.

Создадим модель по исходным формулам для предсказания значений по неизменённым признакам:

## Проверка алгоритма:

Разделю дату на таргет и признаки:

In [8]:
target = df['Страховые выплаты']
features = df.drop(['Страховые выплаты'], axis = 1)

Разделю получившееся на тестовую и тренировочную выборки:

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=254)

Посчитаем R2 у данных без преобразований:

In [10]:
#Качество обучения модели без шифрования:
model = LinearRegression()
model.fit(features_train, target_train)
R2_original = r2_score(target_test, model.predict(features_test))
print("R2 =", R2_original)

R2 = 0.41668703426036524


Создадим рандомную матрицу, размер возьмем нашей матрицы благодаря методу shape.

In [11]:
#Узнаем размер нашей матрицы:
m = features.shape[1]
#Создаем рандомную:
random_m = np.random.randint(1,10, size = (m,m))

In [12]:
random_m

array([[8, 7, 9, 9],
       [6, 5, 4, 6],
       [7, 4, 9, 4],
       [9, 3, 4, 2]])

In [13]:
#Проверим ее на обратимость:
np.linalg.inv(random_m)

array([[ 0.49382716, -0.60493827, -0.39506173,  0.38271605],
       [-2.69135802,  3.24691358,  1.75308642, -1.13580247],
       [ 0.04938272, -0.16049383,  0.16049383, -0.0617284 ],
       [ 1.71604938, -1.82716049, -1.17283951,  0.60493827]])

Матрица обратима. Можно умножать!

In [14]:
encrypted_m = features.dot(random_m)
encrypted_m

,0,1,2,3
0,347463.0,198615.0,446577.0,198657.0
1,266285.0,152233.0,342188.0,152278.0
2,147174.0,84145.0,189116.0,84174.0
3,292044.0,166911.0,375392.0,166930.0
4,182876.0,104547.0,235021.0,104577.0
...,...,...,...,...
4995,250086.0,142946.0,321420.0,142972.0
4996,367013.0,209773.0,471740.0,209806.0
4997,237438.0,135706.0,305188.0,135724.0
4998,229067.0,130926.0,294409.0,130947.0


In [15]:
features_train, features_test, target_train, target_test = train_test_split(encrypted_m, target, test_size=0.25, random_state=254)

Все необходимые преобразования сделали, теперь посчитаем R2 и сравним ее:

In [16]:
#Качество обучения модели после шифрования:
model = LinearRegression()
model.fit(features_train, target_train)
R2_encrypted = r2_score(target_test, model.predict(features_test))
print("R2 =", R2_encrypted)

R2 = 0.4166870342604748


In [17]:
display(R2_original)
display(R2_encrypted)

0.41668703426036524

0.4166870342604748

Вывод: качество обучения модели не изменилось после шифрования данных методом умножения датасета на случайную обратимую матрицу.